<a href="https://colab.research.google.com/github/GasanaElysee12/Assignment-of-Natural-Language-Processing/blob/main/Lab2_intro_to_wordvectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 2: Introduction to wordvectors </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [ ]:
import io, sys
import numpy as np

In [ ]:
def load_vectors(filename):
    fin = io.open(filename, 'r', encoding='utf-8', newline='\n')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])
    return data

In [ ]:
# Loading word vectors

print('')
print(' ** Word vectors ** ')
print('')

'''
word_vectors is a dictionary that maps words to their numerical word vector
[word (string)] = [np-array] 
'''
word_vectors = load_vectors('wiki.en.vec')

tree_vector = word_vectors['tree']
print(type(tree_vector), len(tree_vector))


 ** Word vectors ** 

<class 'numpy.ndarray'> 300


In [ ]:
## This function computes the cosine similarity between vectors u and v

def cosine(u, v):
    '''
    Parameters:
    u : 1-D numpy array
    v : 1-D numpy array 
    
    Returns:
    cos (float) : value of the cosine similairy between vectors u, v 
    '''

    ## FILL CODE
    cos=np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(v))
    return cos 


In [ ]:
# compute similarity between words

print('similarity(apple, apples) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['apples']))
print('similarity(apple, banana) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['banana']))
print('similarity(apple, tiger) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['tiger']))

similarity(apple, apples) = 0.637
similarity(apple, banana) = 0.431
similarity(apple, tiger) = 0.212


In [ ]:
## Functions for nearest neighbor
## This function returns the word corresponding to 
## nearest neighbor vector of x
## The list exclude_words can be used to exclude some
## words from the nearest neighbors search

def nearest_neighbor(x, word_vectors, exclude_words=[]):
    '''
    Parameters:
    x (string): word to find its nearest neighbour 
    word_vectors (Python dict): {word (string): np-array of word vector}
    exclude_words (list of strings): words to be excluded from the search
    
    Returns:
    best_word (string) : the word whose word vector is the nearest neighbour 
    to the word vector of x
    '''
    best_score = -1.0
    best_word = None

    ## FILL CODE
    for label,vector in word_vectors.items():
        scores=cosine(x,vector)
        if scores>best_score and label not in exclude_words:
            best_score=scores
            best_word=label
            
    return best_word

In [ ]:
print('')
print('The nearest neighbor of cat is: ' +
      nearest_neighbor(word_vectors['cat'], word_vectors, exclude_words = ['cat', 'cats']))


The nearest neighbor of cat is: dog


#### Hint (using python priorty queues with the heapq datastructure): 
if you don't want to store all the words and scores you can use the priortiy queue and only store the best K element so far. 

In [ ]:

## This function return the words corresponding to the
## K nearest neighbors of vector x.
## You can use the functions heappush and heappop.

def knn(x, vectors, k):
    '''
    Parameters:
    x (string): word to find its nearest neighbour 
    vectors (Python dict): {word (string): np-array of word vector}
    k (int): number of nearest neighbours to be found
    
    Returns: 
    k_nearest_neighbors (list of tuples): [(score, word), (score, word), ....]
    '''

    k_nearest_neighbors = None
    score=0.0
    
    list_score=[]
    ## FILL CODE
    for label, vector in vectors.items():
        score=cosine(x,vector)
        list_score.append((score,label))
    # print(list_score.values())   
    list_score.sort(key=lambda x: x[0],reverse=True)
    #Since the first is the cat, we don't need it. Propose is
    # to start at k=1 and ends at k+1 since we need k nearest-neighbors
    k_nearest_neighbors=list_score[1:k+1]
        
        
        
    return k_nearest_neighbors

In [ ]:
knn_cat = knn(word_vectors['cat'], word_vectors, 5)
print('')
print('cat')
print('--------------')
for score, word in knn(word_vectors['cat'], word_vectors, 5):
    print (word + '\t%.3f' % score)



cat
--------------
cats	0.732
dog	0.638
pet	0.573
rabbit	0.549
dogs	0.538


#### Hint: 
To find the analogies, we find the nearest neighbour associated with the wordvector d
$$ d = \frac{c}{\Vert {c} \Vert} + \frac{b}{\Vert {b} \Vert} - \frac{a}{\Vert {a} \Vert}$$


In [ ]:
## This function return the words d, such that a:b and c:d
## verifies the same relation

def analogy(a, b, c, word_vectors):
    '''
    Parameters:
    a (string): word a
    b (string): word b
    c (string): word c
    word_vectors (Python dict): {word (string): np-array of word vector}
    
    Returnrs: 
    the word d (string) associated with c such that c:d is similar to a:b 
    
    '''
    
    ## FILL CODE

    
    vector_a=word_vectors[a]
    vector_b=word_vectors[b]
    vector_c=word_vectors[c]
    
    #Normalize the vectors 
    
    vector_a_norm=vector_a/np.linalg.norm(vector_a)
    vector_b_norm=vector_b/np.linalg.norm(vector_b)
    vector_c_norm=vector_c/np.linalg.norm(vector_c)
        
    #Associated word to a.
    result=nearest_neighbor((vector_b_norm+vector_c_norm)-vector_a_norm,word_vectors,[a,b,c])

    return result

In [ ]:
# Word analogies

print('')
print('france - paris + rome = ' + analogy('paris', 'france', 'rome', word_vectors))


france - paris + rome = italy


## A word about biases in word vectors

In [ ]:
## A word about biases in word vectors:

print('')
print('similarity(genius, man) = %.3f' %
      cosine(word_vectors['man'], word_vectors['genius']))
print('similarity(genius, woman) = %.3f' %
      cosine(word_vectors['woman'], word_vectors['genius']))


similarity(genius, man) = 0.445
similarity(genius, woman) = 0.325


In [ ]:
## Compute the association strength between:
##   - a word w
##   - two sets of attributes A and B

def association_strength(w, A, B, vectors):
    '''
    Parameters:
    w (string): word w
    A (list of strings): The words belonging to set A
    B (list of strings): The words belonging to set B
    vectors (Python dict): {word (string): np-array of word vector}
    
    Returnrs: 
    strength (float): the value of the association strength 
    '''
    
    strength = 0.0
    part_a = 0.0
    part_b = 0.0 
    
    ## FILL CODE
    for first_word in A:
        part_a +=cosine(vectors[w],vectors[first_word])
    part_a/=len(A)
    for second_word in B:
        part_b +=cosine(vectors[w],vectors[second_word]) 
    part_b/=len(B)
    
    strength=part_a- part_b
    
    

    return strength

In [ ]:
## Perform the word embedding association test between:
##   - two sets of words X and Y
##   - two sets of attributes A and B

def weat(X, Y, A, B, vectors):
    '''
    Parameters:
    X (list of strings): The words belonging to set X
    Y (list of strings): The words belonging to set Y
    A (list of strings): The words belonging to set A
    B (list of strings): The words belonging to set B
    vectors (Python dict): {word (string): np-array of word vector}
    
    Returns: 
    score (float): the value of the group association strength  
    '''
    
    score = 0.0
    
    first_association=0.0
    second_association=0.0
    
    
    ## FILL CODE
    for x in X:
        first_association +=association_strength(x,A,B,vectors)
        
    for y in Y:
        second_association +=association_strength(y,A, B, vectors)
    score=first_association-second_association
        

    return score

## Word translation using word vectors

In the following, we will use word vectors in English and French to translate words from English to French. The idea is to learn a linear function that maps English word vectors to their correponding French word vectors. To learn this linear mapping, we will use a small bilingual lexicon, that contains pairs of words in English and French that are translations of each other.

The following function will load the small English-French bilingual lexicon:

In [ ]:
## Replicate one of the experiments from:
##
## Semantics derived automatically from language corpora contain human-like biases
## Caliskan, Bryson, Narayanan (2017)

career = ['executive', 'management', 'professional', 'corporation', 
          'salary', 'office', 'business', 'career']
family = ['home', 'parents', 'children', 'family',
          'cousins', 'marriage', 'wedding', 'relatives']
male = ['john', 'paul', 'mike', 'kevin', 'steve', 'greg', 'jeff', 'bill']
female = ['amy', 'joan', 'lisa', 'sarah', 'diana', 'kate', 'ann', 'donna']

print('')
print('Word embedding association test: %.3f' %
      weat(career, family, male, female, word_vectors))


Word embedding association test: 0.847


In [ ]:
def load_lexicon(filename):
    '''
    Parameters:
    filename(string): the path of the lexicon
    
    Returns:
    data(list of pairs of string): the bilingual lexicon
    '''
    fin = io.open(filename, 'r', encoding='utf-8', newline='\n')
    data = []
    for line in fin:
        a, b = line.rstrip().split(' ')
        data.append((a, b))
    return data

In [ ]:
word_vectors_en = load_vectors('wiki.en.vec')
word_vectors_fr = load_vectors('wiki.fr.vec')
lexicon = load_lexicon("lexicon-en-fr.txt")
print(lexicon[:5])

[('the', 'le'), ('the', 'les'), ('the', 'la'), ('and', 'et'), ('was', 'fut')]


In [ ]:
# We split the lexicon into a train and validation set
train = lexicon[:5000]
valid = lexicon[5000:5100]

The following function will learn the mapping from English to French. The idea is to build two matrices $X_{\text{en}}$ and $X_{\text{fr}}$, and to find a mapping $M$ that minimizes $||X_{\text{en}} W - X_{\text{fr}} ||_2$. In numpy, this mapping can be obtained by using the `numpy.linalg.lstsq` function.

In [ ]:
def align(word_vectors_en, word_vectors_fr, lexicon):
    '''
    Parameters:
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    lexicon(list of pairs of string): bilingual training lexicon
    
    Returns
    mapping(np.array): the mapping from English to French vectors
    '''
    x_en, x_fr = [], []
    
    ## FILL CODE
    #Unpack dictionaries for eenglish and fresh

    #Loop into lexicon
    for word_eng,word_fr in lexicon:
        eng_words=word_vectors_en[word_eng]
        fr_words=word_vectors_fr[word_fr ]
        #make a matrix for english and fresh documents
        x_en.append(eng_words)
        x_fr.append(fr_words)
    
   
    #Initialize weight to in order to minimize the euclidean distance
   
    M=np.linalg.lstsq(x_en,x_fr)
    
      
    
     

    
    return M[0]

In [ ]:
mapping = align(word_vectors_en, word_vectors_fr, train)
mapping

/var/folders/_b/0hmhgmr17gzcn7j7_yqdx3440000gn/T/ipykernel_18511/310793915.py:27: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  M=np.linalg.lstsq(x_en,x_fr)


array([[-0.06215271, -0.00061612,  0.00495882, ...,  0.00319385,
         0.00671244, -0.02478187],
       [-0.00477066, -0.03701747,  0.08473041, ..., -0.00392391,
        -0.04316016,  0.02946261],
       [-0.01878439,  0.00219259,  0.01288385, ...,  0.02232575,
        -0.01646909,  0.00014238],
       ...,
       [ 0.07147157,  0.0052355 ,  0.03815085, ..., -0.00191704,
         0.08435404, -0.00642549],
       [ 0.00620476, -0.01829007,  0.00626264, ..., -0.00939435,
        -0.04843014,  0.07418728],
       [ 0.01596265, -0.01573741, -0.0473514 , ..., -0.00421677,
         0.01319889,  0.01050039]])

Given a mapping, a set of word English word vector and French word vectors, the next function will translate the English word to French. To do so, we apply the mapping on the English word, and retrieve the nearest neighbor of the obtained vector in the set of French word vectors. The translation is then the corresponding French word.

In [ ]:
def translate(word, word_vectors_en, word_vectors_fr, mapping):
    '''
    Parameters:
    word(string): an English word
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    mapping(np.array): the mapping from English to French vectors
    
    Returns
    A string containing the translation of the English word
    '''
    
    ## FILL CODE
    eng_set=word_vectors_en[word]
    fra_set=np.dot(eng_set,mapping)
    word_translated=nearest_neighbor(fra_set,word_vectors_fr)
    
 
    

    return word_translated

In [ ]:
print(translate("world", word_vectors_en, word_vectors_fr, mapping))
print(translate("machine", word_vectors_en, word_vectors_fr, mapping))
print(translate("learning", word_vectors_en, word_vectors_fr, mapping))

monde
machine
apprentissage


Finally, let's implement a function to evaluate this method on the validation lexicon:

In [ ]:
def evaluate(valid, word_vectors_en, word_vectors_fr, mapping):
    '''
    Parameters:
    valid(a list of pairs of string): the validation lexicon
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    mapping(np.array): the mapping from English to French vectors
    
    Returns
    Accuracy(float): the accuracy on the validation lexicon
    '''
    acc, n = 0.0, 0
    
    ## FILL CODE
    for word, trans in valid: 
        if trans == translate(word, word_vectors_en, word_vectors_fr, mapping):
            acc +=1.0
        n +=1
        
      

    return acc*100/n

In [ ]:
evaluate(valid, word_vectors_en, word_vectors_fr, mapping)

64.0